In [1]:
import tensorflow.compat.v1 as tf
from tensorflow import feature_column
from tensorflow.keras import layers
tf.disable_v2_behavior()
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

%matplotlib inline

Instructions for updating:
non-resource variables are not supported in the long term


In [4]:
seed = 0
# a = tf.Variable(tf.random_normal([1], dtype=tf.float64, seed=0))
# b = tf.Variable(tf.random_normal([1], dtype=tf.float64, seed=0))

# tensorflow, numpy 랜덤 값을 설정합니다.
tf.set_random_seed(seed)
np.random.seed(seed)

In [9]:
#doginfo.csv파일 데이터를 pandas를 이용해 읽어옵니다.
dog_data=pd.read_csv("doginfo.csv")
print(dog_data.head())

  kindCd  kindNum neuterYn sexCd  weight  noticeDays  age2  processState  \
0     푸들    128.0        N     F    7.46          10    12             0   
1    믹스견    114.0        N     M    7.00          14     1             1   
2    믹스견    114.0        U     M    4.50          11     2             0   
3    진도견     67.0        N     M   10.00           8     1             0   
4    믹스견    114.0        N     M    6.00           8     4             0   

   Unnamed: 8   kindCd.1  kindNum.1  
0         NaN    골든 리트리버       54.0  
1         NaN    그레이 하운드       56.0  
2         NaN     그레이트 덴       55.0  
3         NaN  그레이트 피레니즈      118.0  
4         NaN         기타      115.0  


In [10]:
# dog_data.loc[dog_data["neuterYn"] == "N", "neuterYn"] = 0
# dog_data.loc[dog_data["neuterYn"] == "U", "neuterYn"] = 1
# dog_data.loc[dog_data["neuterYn"] == "Y", "neuterYn"] = 2

# dog_data.loc[dog_data["sexCd"] == "M", "sexCd"] = 0
# dog_data.loc[dog_data["sexCd"] == "F", "sexCd"] = 1
# dog_data.loc[dog_data["sexCd"] == "Q", "sexCd"] = 2

# del dog_data['kindCd']
# del dog_data['kindCd.1']

dog_data.drop(['kindCd', 'neuterYn', 'sexCd', 'Unnamed: 8', 'kindCd.1', 'kindNum.1'], axis='columns', inplace=True)


data=np.array(dog_data, dtype=np.float64)


print(data)
print(type(data))

[[128.     7.46  10.    12.     0.  ]
 [114.     7.    14.     1.     1.  ]
 [114.     4.5   11.     2.     0.  ]
 ...
 [114.     1.    10.     0.     1.  ]
 [128.     6.    12.     3.     0.  ]
 [114.     3.5   10.     0.     0.  ]]
<class 'numpy.ndarray'>


In [13]:
#numpy 배열에서 데이터 변화요인(kindCd, neuterYn, sexCd, weight, noticeDays, age2)으로 사용할 데이터를 뽑아냅니다.
xData=data[:, :4]
print(xData)

[[128.     7.46  10.    12.  ]
 [114.     7.    14.     1.  ]
 [114.     4.5   11.     2.  ]
 ...
 [114.     1.    10.     0.  ]
 [128.     6.    12.     3.  ]
 [114.     3.5   10.     0.  ]]


In [15]:
#numpy배열에서 결과(입양여부)로 사용할 데이터를 뽑아냅니다.
yData=data[:,[4]]
print(yData)

[[0.]
 [1.]
 [0.]
 ...
 [1.]
 [0.]
 [0.]]


In [18]:
#뽑아낸 데이터를 tensorflow로 처리하기 위해 placeholder를 만듭니다.
X=tf.placeholder(dtype=tf.float64, shape=[None, 4])
Y=tf.placeholder(dtype=tf.float64, shape=[None, 1])
#다변인 선형 회귀 모델의 기울기와 y절편을 임의의 값으로 초기화합니다.
a=tf.Variable(tf.random_uniform([4,1], dtype=tf.float64))#6행 1열 난수를 발생시킵니다.
b=tf.Variable(tf.random_uniform([1], dtype=tf.float64)) #1행 1열 난수를 발생시킵니다.


In [19]:
y=tf.sigmoid(tf.matmul(X, a)+b)
loss=-tf.reduce_mean(Y*tf.log(y)+(1-Y)*tf.log(1-y))
gradient_descent=tf.train.GradientDescentOptimizer(0.001).minimize(loss)

In [21]:
predicted=tf.cast(y>=0.5, dtype=tf.float64)
accuracy=tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float64))

In [22]:
with tf.Session() as sess:  # 세션 구간을 만듦
    sess.run(tf.global_variables_initializer())
    print(sess.run(a))
    print(sess.run(b))
    for i in range(3001):
        # count : 학습 횟수, a1 : 기울기(공부 시간), a2 : 기울기(과외 시간), b : y절편, loss : 오차 함수
        a_, b_, loss_, _ = sess.run([a, b, loss, gradient_descent], feed_dict = {X:xData, Y:yData})
        if i % 300 == 0:
            print('count : {}, a1 = {}, a2 = {}, a3 = {}, a4 = {}, b = {}, loss = {}'.format(i, a_[0], a_[1], a_[2], a_[3], b_, loss_)) # 학습시킨 데이터 출력
            #print('loss={}, print={}'.format(loss_, b_))
# 학습 완료 -----------------------------------------------------------------------------------------------------
# 테스트 데이터

#     new_x = np.array([6, 5]).reshape(1, 2)  # reshape() 함수를 이용해서 1행 2열인 2차원 numpy 배열을 만듭니다.
    
#     # sigmoid() 함수 연산 결과에 테스트 데이터를 대입시켜 연산합니다.
#     result, new_y = sess.run([predicted, y], feed_dict = {X:new_x})
#     print('공부 시간 : %d, 과외 시간 : %d' % (new_x[:, 0], new_x[:, 1]))
#     print('합격 여부(sigmoid() 함수 실행 결과를 0.5 이상은 1, 미만은 0으로 반환) : %d, 합격 확률(sigmoid() 함수 실행 결과) : %f)' % (result, new_y))
    
#     for i in range(11):
#         new_x = np.array([0, i]).reshape(1, 2)  # 과외 시간만 계속 늘림
#         result, new_y = sess.run([predicted, y], feed_dict={X:new_x})
#         print('공부 시간 : %d, 과외 시간 : %2d' % (new_x[:, 0], new_x[:, 1]), end=',')
#         print('합격 여부 : %s, 합격 확률 : %8.4f%%' % ('합격' if result == 1 else '불합격', new_y*100))

[[0.64783187]
 [0.83705062]
 [0.09867108]
 [0.58890727]]
[0.69447269]
count : 0, a1 = [nan], a2 = [nan], a3 = [nan], a4 = [nan], b = [nan], loss = nan
count : 300, a1 = [nan], a2 = [nan], a3 = [nan], a4 = [nan], b = [nan], loss = nan
count : 600, a1 = [nan], a2 = [nan], a3 = [nan], a4 = [nan], b = [nan], loss = nan
count : 900, a1 = [nan], a2 = [nan], a3 = [nan], a4 = [nan], b = [nan], loss = nan
count : 1200, a1 = [nan], a2 = [nan], a3 = [nan], a4 = [nan], b = [nan], loss = nan
count : 1500, a1 = [nan], a2 = [nan], a3 = [nan], a4 = [nan], b = [nan], loss = nan
count : 1800, a1 = [nan], a2 = [nan], a3 = [nan], a4 = [nan], b = [nan], loss = nan
count : 2100, a1 = [nan], a2 = [nan], a3 = [nan], a4 = [nan], b = [nan], loss = nan
count : 2400, a1 = [nan], a2 = [nan], a3 = [nan], a4 = [nan], b = [nan], loss = nan
count : 2700, a1 = [nan], a2 = [nan], a3 = [nan], a4 = [nan], b = [nan], loss = nan
count : 3000, a1 = [nan], a2 = [nan], a3 = [nan], a4 = [nan], b = [nan], loss = nan
